In [1]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from mcmodels.core import Mask
import os
import allensdk.core.json_utilities as ju
from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache
from mcmodels.core import VoxelModelCache
from matplotlib.pyplot import gcf
import matplotlib.patches as mpatches
import seaborn as sns

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.linear_model.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.linear_model. Anything that cannot be imported from sklearn.linear_model is now part of the private API.
  warnings.warn(message, FutureWarning)
/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)


In [4]:

def fix_pdcsv(csv):
    
    #wt_conn = pd.read_csv('/Users/samsonkoelle/wt_leafsmooth_leafsurface_sumtarget.csv',  header=[0,1])
    csv_rownames = np.asarray(csv.iloc[:,0])
    csv = csv.iloc[:,1:]
    csv.index = csv_rownames
    return(csv)

def fill_df_na(data, indices):
    
    for i in range(indices.shape[0]):
        data.iloc[indices[i,0],indices[i,1]] = np.nan
        
    return(data)



In [5]:
major_structures = ['CB',
 'CTXsp',
 'HPF',
 'HY',
 'Isocortex',
 'MB',
 'MY',
 'OLF',
 'P',
 'PAL',
 'STR',
 'TH']

wt_conn = pd.read_csv('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/paper/connectivities/elsumsurfleafsmth_v3_leafsum_wt.csv', header = [0,1])
wt_conn = fix_pdcsv(wt_conn)

#load data
dists = pd.read_csv('/Users/samsonkoelle/alleninstitute/sambranch/mouse_connectivity_models/analyses/paper/getinfo/distances.csv',  header=[0,1])
dists = fix_pdcsv(dists)

nonzero_ind = np.where(np.sum(wt_conn, axis = 1) > 0)[0]
wt_conn_nonzero = wt_conn.iloc[nonzero_ind]
wt_conn_nonzero_log = np.log10(wt_conn_nonzero)
wt_conn_nonzero_log.replace([-np.inf], -5, inplace=True)
wt_conn_nonzero_log[wt_conn_nonzero_log < -5] = -5
#sns.heatmap(wt_conn_nonzero_log, vmin = -5, vmax = 0)

/Users/samsonkoelle/anaconda3/envs/allen_010719_5/lib/python3.7/site-packages/ipykernel_launcher.py:23: RuntimeWarning: divide by zero encountered in log10


In [6]:
TOP_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
INPUT_JSON = os.path.join(TOP_DIR, 'input_011520.json')
EXPERIMENTS_EXCLUDE_JSON = os.path.join(TOP_DIR, 'experiments_exclude.json')
FILE_DIR = '/Users/samsonkoelle/alleninstitute/mcm_2020/mcm_updated/'
OUTPUT_DIR = os.path.join(FILE_DIR, 'output')

mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
ai_map = st.get_id_acronym_map()
ia_map = {value: key for key, value in ai_map.items()}


input_data = ju.read(INPUT_JSON)
manifest_file = input_data.get('manifest_file')
manifest_file = os.path.join(TOP_DIR, manifest_file)
experiments_exclude = ju.read(EXPERIMENTS_EXCLUDE_JSON)

#its unclear why the hyperparameters are loaded from the output directory
cache = VoxelModelCache(manifest_file=manifest_file)

columns = np.asarray(list(wt_conn_nonzero.columns))
shapes = np.zeros(wt_conn_nonzero_log.shape[1])
for i in range(wt_conn_nonzero_log.shape[1]):
    print(i)
    iii = columns[i][1]
    hemi = columns[i][0]
    if hemi == 'ipsi':
        mask = Mask.from_cache(cache,structure_ids=[ai_map[iii]],hemisphere_id=2)
    if hemi == 'contra':
        mask = Mask.from_cache(cache,structure_ids=[ai_map[iii]],hemisphere_id=1)
    shapes[i] = mask.coordinates.shape[0]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [7]:
wt_conn_density = wt_conn / shapes

In [8]:
wt_conn_density

ipsi                                                          \
                 FRP           MOp           MOs         SSp-n       SSp-bfd   
FRP1             NaN           NaN           NaN           NaN           NaN   
FRP2/3  3.003245e-04  4.724258e-06  5.377601e-05  3.099090e-07  2.290292e-08   
FRP5    1.760610e-04  6.829191e-06  2.663268e-05  5.816475e-07  5.143612e-08   
FRP6a   2.250595e-04  2.518775e-07  1.516770e-05  1.613074e-08  2.536519e-09   
FRP6b            NaN           NaN           NaN           NaN           NaN   
...              ...           ...           ...           ...           ...   
FL               NaN           NaN           NaN           NaN           NaN   
FN      0.000000e+00  1.670691e-08  7.376494e-09  1.160768e-08  9.734414e-09   
IP      3.917643e-08  5.588792e-08  1.584557e-08  2.055302e-07  1.100229e-07   
DN               NaN           NaN           NaN           NaN           NaN   
VeCB             NaN           NaN           NaN           NaN           NaN   

                                                                              \
              SSp-ll         SSp-m        SSp-ul        SSp-tr        SSp-un   
FRP1             NaN           NaN           NaN           NaN           NaN   
FRP2/3  9.417345e-09  4.408117e-07  8.469349e-08  6.035230e-09  1.281565e-08   
FRP5    3.367318e-08  1.809108e-06  9.770360e-07  2.149634e-08  3.496951e-07   
FRP6a   0.000000e+00  3.762084e-07  2.316045e-08  0.000000e+00  1.068770e-08   
FRP6b            NaN           NaN           NaN           NaN           NaN   
...              ...           ...           ...           ...           ...   
FL               NaN           NaN           NaN           NaN           NaN   
FN      7.379315e-13  2.001958e-08  2.687091e-09  9.258872e-12  5.887014e-12   
IP      1.844539e-08  1.798992e-07  3.616937e-08  1.882546e-08  1.918660e-08   
DN               NaN           NaN           NaN           NaN           NaN   
VeCB             NaN           NaN           NaN           NaN           NaN   

        ...        contra                                            \
        ...           SIM            AN           PRM          COPY   
FRP1    ...           NaN           NaN           NaN           NaN   
FRP2/3  ...  0.000000e+00  0.000000e+00  3.655607e-10  0.000000e+00   
FRP5    ...  2.109935e-08  3.199841e-08  3.336124e-08  8.910706e-08   
FRP6a   ...  2.116045e-10  3.866904e-11  1.546168e-10  0.000000e+00   
FRP6b   ...           NaN           NaN           NaN           NaN   
...     ...           ...           ...           ...           ...   
FL      ...           NaN           NaN           NaN           NaN   
FN      ...  7.165093e-07  1.477115e-06  2.303113e-06  2.747002e-06   
IP      ...  1.211188e-08  2.186079e-08  1.232382e-07  6.084999e-07   
DN      ...           NaN           NaN           NaN           NaN   
VeCB    ...           NaN           NaN           NaN           NaN   

                                                                              \
                 PFL            FL            FN            IP            DN   
FRP1             NaN           NaN           NaN           NaN           NaN   
FRP2/3  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
FRP5    1.001071e-07  7.346424e-08  1.809865e-08  7.328739e-10  1.092451e-10   
FRP6a   1.976423e-08  0.000000e+00  0.000000e+00  0.000000e+00  0.000000e+00   
FRP6b            NaN           NaN           NaN           NaN           NaN   
...              ...           ...           ...           ...           ...   
FL               NaN           NaN           NaN           NaN           NaN   
FN      3.474649e-08  1.953683e-08  3.897305e-05  1.649267e-05  1.869513e-06   
IP      7.717375e-07  2.318500e-08  2.069809e-07  7.227000e-08  2.083779e-07   
DN               NaN           NaN           NaN           NaN           NaN   
VeCB             NaN           NaN           NaN  